In [1]:
import numpy as np

# Two-body model
## 2 Ligands

In [2]:
x = np.array([6, 5, 4, 4, 3, 3, 2, 2, 1, 0])
N = np.stack([x, 6 - x], axis=1)
print(N)

[[6 0]
 [5 1]
 [4 2]
 [4 2]
 [3 3]
 [3 3]
 [2 4]
 [2 4]
 [1 5]
 [0 6]]


In [3]:
np.linalg.matrix_rank(N)

2

# Three-body model

## 2 Ligands

In [60]:
N_2 = np.array(
    [
        # [6, 0, 12, 0, 0, 3, 0, 0],  # homoleptic
        [5, 1, 8, 4, 0, 2, 1, 0],  # 5+1
        [4, 2, 5, 6, 1, 1, 2, 0],  # 4+2 cis
        [4, 2, 4, 8, 0, 2, 0, 1],  # 4+2 trans
        [3, 3, 3, 6, 3, 0, 3, 0],  # 3+3 fac
        [3, 3, 2, 8, 2, 1, 1, 1],  # 3+3 mer
        [2, 4, 1, 6, 5, 0, 2, 1],  # 4+2 cis
        [2, 4, 0, 8, 4, 1, 0, 2],  # 4+2 trans
        [1, 5, 0, 4, 8, 0, 1, 2],  # 5+1
        # [0, 6, 0, 0, 12, 0, 0, 3],  # homoleptic
    ]
)
# Assert that N_A + N_B = 6
np.testing.assert_equal(np.sum(N_2[:, :2], axis=1), 6 * np.ones(len(N_2)))
# Assert that N^cis_AA + N^cis_AB + N^cis_BB = 12
np.testing.assert_equal(np.sum(N_2[:, 2:5], axis=1), 12 * np.ones(len(N_2)))
# Assert that N^trans_AA + N^trans_AB + N^trans_BB = 3
np.testing.assert_equal(np.sum(N_2[:, 5:], axis=1), 3 * np.ones(len(N_2)))

# Cis terms
np.testing.assert_equal(2 * N_2[:, 2], 4 * N_2[:, 0] - N_2[:, 3])  # AA
np.testing.assert_equal(2 * N_2[:, 4], 4 * N_2[:, 1] - N_2[:, 3])  # BB
# Trans terms
np.testing.assert_equal(2 * N_2[:, 5], N_2[:, 0] - N_2[:, 6])  # AA
np.testing.assert_equal(2 * N_2[:, 7], N_2[:, 1] - N_2[:, 6])  # BB

print(np.linalg.matrix_rank(N_2))

4


## 3 Ligands
$[q_A, q_B, q_C, q^{cis}_{AA}, q^{cis}_{BB}, q^{cis}_{CC}, q^{cis}_{AB}, q^{cis}_{AC}, q^{cis}_{BC}, q^{trans}_{AA}, q^{trans}_{BB}, q^{trans}_{CC}, q^{trans}_{AB}, q^{trans}_{AC}, q^{trans}_{BC}] $  

In [87]:
def swap_AB(vec):
    return [vec[i] for i in (1, 0, 2, 4, 3, 5, 6, 8, 7, 10, 9, 11, 12, 14, 13)]


def swap_AC(vec):
    return [vec[i] for i in (2, 1, 0, 5, 4, 3, 8, 7, 6, 11, 10, 9, 14, 13, 12)]


def swap_BC(vec):
    return swap_AB(swap_AC(swap_AB(vec)))


np.testing.assert_equal(
    swap_AB([4, 1, 1, 5, 0, 0, 3, 3, 1, 1, 0, 0, 1, 1, 0]),
    [1, 4, 1, 0, 5, 0, 3, 1, 3, 0, 1, 0, 1, 0, 1],
)
np.testing.assert_equal(
    swap_AB([4, 1, 1, 4, 0, 0, 4, 4, 0, 2, 0, 0, 0, 0, 1]),
    [1, 4, 1, 0, 4, 0, 4, 0, 4, 0, 2, 0, 0, 1, 0],
)
np.testing.assert_equal(
    swap_AC([4, 1, 1, 5, 0, 0, 3, 3, 1, 1, 0, 0, 1, 1, 0]),
    [1, 1, 4, 0, 0, 5, 1, 3, 3, 0, 0, 1, 0, 1, 1],
)

In [89]:
N_3 = np.array(
    [
        # 4 A + 1 B + 1 C
        [4, 1, 1, 5, 0, 0, 3, 3, 1, 1, 0, 0, 1, 1, 0],  # CA
        [4, 1, 1, 4, 0, 0, 4, 4, 0, 2, 0, 0, 0, 0, 1],  # TA
        # 1 A + 4 B + 1 C
        swap_AB([4, 1, 1, 5, 0, 0, 3, 3, 1, 1, 0, 0, 1, 1, 0]),  # CA
        swap_AB([4, 1, 1, 4, 0, 0, 4, 4, 0, 2, 0, 0, 0, 0, 1]),  # TA
        # 1 A + 1 B + 4 C
        swap_AC([4, 1, 1, 5, 0, 0, 3, 3, 1, 1, 0, 0, 1, 1, 0]),  # CA
        swap_AC([4, 1, 1, 4, 0, 0, 4, 4, 0, 2, 0, 0, 0, 0, 1]),  # TA
        # 3 A + 2 B + 1 C
        [3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0],  # FA
        [3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0],  # MAT
        [3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1],  # MAC
        # 3 A + 1 B + 2 C
        swap_AC([3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0]),  # FA
        swap_AC([3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0]),  # MAT
        swap_AC([3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1]),  # MAC
        # 2 A + 3 B + 1 C
        swap_AB([3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0]),  # FA
        swap_AB([3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0]),  # MAT
        swap_AB([3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1]),  # MAC
        # 2 A + 1 B + 3 C
        swap_AC(swap_AB([3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0])),  # FA
        swap_AC(swap_AB([3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0])),  # MAT
        swap_AC(swap_AB([3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1])),  # MAC
        # 1 A + 2 B + 3 C
        swap_AC([3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0]),  # FA
        swap_AC([3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0]),  # MAT
        swap_AC([3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1]),  # MAC
        # 1 A + 3 B + 2 C
        swap_AC(swap_AB([3, 2, 1, 3, 1, 0, 4, 2, 2, 0, 0, 0, 2, 1, 0])),  # FA
        swap_AC(swap_AB([3, 2, 1, 2, 0, 0, 6, 2, 2, 1, 1, 0, 0, 1, 0])),  # MAT
        swap_AC(swap_AB([3, 2, 1, 2, 1, 0, 5, 3, 1, 1, 0, 0, 1, 0, 1])),  # MAC
        # 2 A + 2 B + 2 C
        [2, 2, 2, 1, 1, 0, 2, 4, 4, 0, 0, 1, 2, 0, 0],  # EA (C trans)
        [2, 2, 2, 1, 0, 1, 4, 2, 4, 0, 1, 0, 0, 2, 0],  # EA (B trans)
        [2, 2, 2, 0, 1, 1, 4, 4, 2, 1, 0, 0, 0, 0, 2],  # EA (A trans)
        [2, 2, 2, 1, 1, 1, 3, 3, 3, 0, 0, 0, 1, 1, 1],  # DCS
        [2, 2, 2, 0, 0, 0, 4, 4, 4, 1, 1, 1, 0, 0, 0],  # DTS
    ]
)
# Assert that the two-body terms sum to 6
np.testing.assert_equal(np.sum(N_3[:, :3], axis=1), 6 * np.ones(len(N_3)))
# Assert that the three-body cis terms sum to 12
np.testing.assert_equal(np.sum(N_3[:, 3:9], axis=1), 12 * np.ones(len(N_3)))
# Assert that the three-body trans terms sum to 12
np.testing.assert_equal(np.sum(N_3[:, 9:], axis=1), 3 * np.ones(len(N_3)))

# Cis terms
np.testing.assert_equal(2 * N_3[:, 3], 4 * N_3[:, 0] - N_3[:, 6] - N_3[:, 7])  # AA
np.testing.assert_equal(2 * N_3[:, 4], 4 * N_3[:, 1] - N_3[:, 6] - N_3[:, 8])  # BB
np.testing.assert_equal(2 * N_3[:, 5], 4 * N_3[:, 2] - N_3[:, 7] - N_3[:, 8])  # CC
# Trans terms
np.testing.assert_equal(2 * N_3[:, 9], N_3[:, 0] - N_3[:, 12] - N_3[:, 13])  # AA
np.testing.assert_equal(2 * N_3[:, 10], N_3[:, 1] - N_3[:, 12] - N_3[:, 14])  # BB
np.testing.assert_equal(2 * N_3[:, 11], N_3[:, 2] - N_3[:, 13] - N_3[:, 14])  # CC

print(np.linalg.matrix_rank(N_3))

9


# Archive

In [90]:
A = np.array(
    [
        [6, 0, 12, 0, 0, 3, 0, 0],
        [5, 1, 8, 4, 0, 2, 1, 0],
        [4, 2, 5, 6, 1, 1, 2, 0],
        [4, 2, 4, 8, 0, 2, 0, 1],
        [3, 3, 3, 6, 3, 0, 3, 0],
        [3, 3, 2, 8, 2, 1, 1, 1],
        [2, 4, 1, 6, 5, 0, 2, 1],
        [2, 4, 0, 8, 4, 1, 0, 2],
        [1, 5, 0, 4, 8, 0, 1, 2],
        [0, 6, 0, 0, 12, 0, 0, 3],
    ]
)
# Assert that N_A + N_B = 6
np.testing.assert_equal(np.sum(A[:, :2], axis=1), 6 * np.ones(10))
# Assert that N^cis_AA + N^cis_AB + N^cis_BB = 12
np.testing.assert_equal(np.sum(A[:, 2:5], axis=1), 12 * np.ones(10))
# Assert that N^trans_AA + N^trans_AB + N^trans_BB = 3
np.testing.assert_equal(np.sum(A[:, 5:], axis=1), 3 * np.ones(10))

print(np.linalg.matrix_rank(A))

4


In [160]:
# Cis/Trans splitting:
cis_trans_split = A[2, :] - A[3, :]
# Fac/Mer splitting
fac_mer_split = A[4, :] - A[5, :]
# Assert they are the same
np.testing.assert_array_equal(cis_trans_split, fac_mer_split)
print(fac_mer_split)

[ 0  0  1 -2  1 -1  2 -1]


In [188]:
curvature_51 = A[1, :] - (5 * A[0, :] + 1 * A[-1, :]) / 6
np.testing.assert_equal(curvature_51, A[-2, :] - (1 * A[0, :] + 5 * A[-1, :]) / 6)
print(curvature_51)

[ 0.   0.  -2.   4.  -2.  -0.5  1.  -0.5]


In [186]:
a = np.array([6, 0, 12, 0, 0, 3, 0, 0]) / 6
b = np.array([0, 6, 0, 0, 12, 0, 0, 3]) / 6

X = np.stack([a, b, fac_mer_split, curvature_51])
B = np.array(
    [
        [6, 0, 0, 0],
        [5, 1, 0, 1],
        [4, 2, 0.2, 1.6],
        [4, 2, -0.8, 1.6],
        [3, 3, 0.6, 1.8],
        [3, 3, -0.4, 1.8],
        [2, 4, 0.2, 1.6],
        [2, 4, -0.8, 1.6],
        [1, 5, 0, 1],
        [0, 6, 0, 0],
    ]
)
print(np.linalg.matrix_rank(B @ X - A))
B @ X - A

4


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -8.88178420e-16,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.22044605e-16,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.44089210e-16,
         0.00000000e+00, -4.44089210e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.44089210e-16,
         0.00000000e+00, -4.44089210e-16, -1.11022302e-16,
         0.00000000e+00, -1.11022302e-16],
       [ 0.00000000e+00,  0.000000

In [96]:
np.linalg.matrix_rank(B[:, :2] @ X[:2, :] - A)

2

In [97]:
fac_mer_split

array([ 0. ,  0. ,  0.5, -1. ,  0.5, -0.5,  1. , -0.5])

In [98]:
A

array([[ 6,  0, 12,  0,  0,  3,  0,  0],
       [ 5,  1,  8,  4,  0,  2,  1,  0],
       [ 4,  2,  5,  6,  1,  1,  2,  0],
       [ 4,  2,  4,  8,  0,  2,  0,  1],
       [ 3,  3,  3,  6,  3,  0,  3,  0],
       [ 3,  3,  2,  8,  2,  1,  1,  1],
       [ 2,  4,  1,  6,  5,  0,  2,  1],
       [ 2,  4,  0,  8,  4,  1,  0,  2],
       [ 1,  5,  0,  4,  8,  0,  1,  2],
       [ 0,  6,  0,  0, 12,  0,  0,  3]])

In [99]:
C = A[(True, True, False, True, False, True, False, True, True, True), :]
np.linalg.matrix_rank(C)

4

In [100]:
np.linalg.matrix_rank(A[:, [True, True, False, False, False, True, True, True]])

3

In [101]:
A_reduced = A[:, [True, True, False, True, False, False, True, False]]

In [102]:
# Cis/Trans splitting:
cis_trans_split = (A_reduced[2, :] - A_reduced[3, :]) / 2
# Fac/Mer splitting
fac_mer_split = (A_reduced[4, :] - A_reduced[5, :]) / 2
# Assert they are the same
np.testing.assert_array_equal(cis_trans_split, fac_mer_split)
print(fac_mer_split)

[ 0.  0. -1.  1.]


In [149]:
curvature_facmer = (A_reduced[4, :] + A_reduced[5, :]) / 2 - (
    3 * A_reduced[0, :] + 3 * A_reduced[-1, :]
) / 6
print(curvature_facmer)
curvature_fac = A_reduced[4, :] - (3 * A_reduced[0, :] + 3 * A_reduced[-1, :]) / 6
print(curvature_fac)
curvature_mer = A_reduced[5, :] - (3 * A_reduced[0, :] + 3 * A_reduced[-1, :]) / 6
print(curvature_mer)
curvature_cistrans = (A_reduced[2, :] + A_reduced[3, :]) / 2 - (
    4 * A_reduced[0, :] + 2 * A_reduced[-1, :]
) / 6
print(curvature_cistrans)
curvature_cis = A_reduced[2, :] - (4 * A_reduced[0, :] + 2 * A_reduced[-1, :]) / 6
print(curvature_cis)
curvature_trans = A_reduced[3, :] - (4 * A_reduced[0, :] + 2 * A_reduced[-1, :]) / 6
print(curvature_trans)
curvature_51 = A_reduced[1, :] - (5 * A_reduced[0, :] + 1 * A_reduced[-1, :]) / 6
print(curvature_51)

[0. 0. 7. 2.]
[0. 0. 6. 3.]
[0. 0. 8. 1.]
[0. 0. 7. 1.]
[0. 0. 6. 2.]
[0. 0. 8. 0.]
[0. 0. 4. 1.]


In [159]:
a = np.array([1, 0, 0, 0])
b = np.array([0, 1, 0, 0])

X = np.stack([a, b, 2 * fac_mer_split / 5, curvature_51 / 5])
B = np.array(
    [
        [6, 0, 0, 0],
        [5, 1, 0, 5],
        [4, 2, 1, 8],
        [4, 2, -4, 8],
        [3, 3, 3, 9],
        [3, 3, -2, 9],
        [2, 4, 1, 8],
        [2, 4, -4, 8],
        [1, 5, 0, 5],
        [0, 6, 0, 0],
    ]
)
print(np.linalg.matrix_rank(B @ X - A_reduced))
B @ X - A_reduced

1


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 4.4408921e-16],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00]])

In [144]:
a = np.array([1, 0, 0, 0])
b = np.array([0, 1, 0, 0])

X = np.stack([a, b, fac_mer_split, curvature_facmer])
B = np.array(
    [
        [6, 0, 0, 0],
        [5, 1, 0, 1],
        [4, 2, 0.4, 1.6],
        [4, 2, -1.6, 1.6],
        [3, 3, 1.2, 1.8],
        [3, 3, -0.8, 1.8],
        [2, 4, 0.4, 1.6],
        [2, 4, -1.6, 1.6],
        [1, 5, 0, 1],
        [0, 6, 0, 0],
    ]
)
print(np.linalg.matrix_rank(B @ X - A_reduced))
B @ X - A_reduced

1


array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 3. , 1. ],
       [0. , 0. , 4.8, 1.6],
       [0. , 0. , 4.8, 1.6],
       [0. , 0. , 5.4, 1.8],
       [0. , 0. , 5.4, 1.8],
       [0. , 0. , 4.8, 1.6],
       [0. , 0. , 4.8, 1.6],
       [0. , 0. , 3. , 1. ],
       [0. , 0. , 0. , 0. ]])

In [145]:
A_reduced @ np.linalg.pinv(X)

array([[ 6.        ,  0.        ,  0.        ,  0.        ],
       [ 5.        ,  1.        , -0.11111111,  0.55555556],
       [ 4.        ,  2.        ,  0.22222222,  0.88888889],
       [ 4.        ,  2.        , -1.77777778,  0.88888889],
       [ 3.        ,  3.        ,  1.        ,  1.        ],
       [ 3.        ,  3.        , -1.        ,  1.        ],
       [ 2.        ,  4.        ,  0.22222222,  0.88888889],
       [ 2.        ,  4.        , -1.77777778,  0.88888889],
       [ 1.        ,  5.        , -0.11111111,  0.55555556],
       [ 0.        ,  6.        ,  0.        ,  0.        ]])